In [ ]:
# Run setup_imports to enable importing from src/
%run ./setup_imports.py

In [ ]:
from qiskit import QuantumCircuit
from qonscious.constraints import PackedCHSHTest
from qonscious.policies import MinimumAcceptableValue
from qonscious.core import IBMSamplerAdapter
from qonscious.core import executor
from dotenv import load_dotenv 
import os
import time
import json
from qiskit_ibm_runtime import QiskitRuntimeService
import pandas as pd
from datetime import datetime

In [ ]:
pli_plus = QuantumCircuit(2, 2)
pli_plus.h(0)
pli_plus.cx(0, 1)
pli_plus.measure([0, 1], [0, 1])
   

In [ ]:
reps = 100  # 
delay_between_reps = 60 * 5  # Every five minutes
introspection_shots = 1024
main_circuit_shots = 1024
entanglement_threshold = 1.9

In [ ]:
def get_backend():
  load_dotenv()
  ibm_token = os.getenv("IBM_QUANTUM_TOKEN")
  service = QiskitRuntimeService(channel="ibm_quantum_platform", token=ibm_token)
  return service.least_busy(operational=True, simulator=False)

ibm_backend_adapter = IBMSamplerAdapter(get_backend())

In [ ]:
def on_pass(backend_adapter, introspection_result):
    entry = {}
    print(f"Entanglement passed with score: {introspection_result['CHSH_score']}")
    entry['entanglement_score'] = introspection_result['CHSH_score']
    entry['introspection'] = introspection_result
    entry['execution'] = backend_adapter.run(pli_plus, shots=main_circuit_shots)
    return entry

def on_fail(backend_adapter, introspection_result):
    entry = {}
    print(f"Skipping main circuit - entanglement score was {introspection_result['CHSH_score']}")
    entry['entanglement_score'] = introspection_result['CHSH_score']
    entry['introspection'] = introspection_result
    return entry


In [ ]:

constraint = PackedCHSHTest(policy=MinimumAcceptableValue(entanglement_threshold))

results = []

for i in range(reps):
    print(f"{i+1}. Running introspection + conditional execution")
    entry =  executor.run_conditionally(
        backend_adapter=ibm_backend_adapter,
        constraint=constraint,
        on_pass=on_pass,
        on_fail=on_fail,
        shots=introspection_shots
    )
    results.append(entry)
    if i < reps - 1:
        time.sleep(delay_between_reps)

In [ ]:
import matplotlib.pyplot as plt

chsh_scores = [entry["introspection"]["CHSH_score"] for entry in results]
plt.figure()
plt.plot(range(len(results)), chsh_scores, marker='o')
plt.title("CHSH Score Over Repetitions")
plt.xlabel("Run Index")
plt.ylabel("CHSH Score")
plt.grid(True)
plt.show()

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt

def parse_utc(s):
    return datetime.fromisoformat(s.replace("Z", "+00:00"))

durations = [
    (parse_utc(entry["execution"]["timestamps"]["running"]) -
     parse_utc(entry["introspection"]["timestamps"]["finished"])).total_seconds()
    for entry in results
]

plt.figure()
plt.plot(range(len(durations)), durations, marker='o', color='orange')
plt.title("Total Delay from Introspection End to Execution Start")
plt.xlabel("Run Index")
plt.ylabel("Duration (s)")
plt.grid(True)
plt.show()

In [ ]:
def phi_plus_fidelity(counts):
    shots = sum(counts.values())
    prob_00 = counts.get("00", 0) / shots
    prob_11 = counts.get("11", 0) / shots
    # For ideal \Phi^+, the off-diagonal terms contribute too, but from counts only, we approximate fidelity
    return prob_00 + prob_11  # coarse classical fidelity estimate

fidelities = [phi_plus_fidelity(entry["execution"]["counts"]) for entry in results]

plt.figure()
plt.plot(range(len(fidelities)), fidelities, marker='o', color='green')
plt.title("Approximate Fidelity to Phi+ State")
plt.xlabel("Run Index")
plt.ylabel("Fidelity Estimate")
plt.grid(True)
plt.ylim(0, 1)
plt.show()


In [ ]:
# Save experiment data in a tabular summary
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
csv_filename = f"experiment_summary_{timestamp}.csv"

df = pd.DataFrame([{
    'introspection_created': r.get('introspection', {}).get('timestamps', {}).get('created'),
    'introspection_running': r.get('introspection', {}).get('timestamps', {}).get('running'),
    'introspection_finished': r.get('introspection', {}).get('timestamps', {}).get('finished'),
    'execution_created': r.get('execution', {}).get('timestamps', {}).get('created'),
    'execution_running': r.get('execution', {}).get('timestamps', {}).get('running'),
    'execution_finished': r.get('execution', {}).get('timestamps', {}).get('finished'),    
    'entanglement_score': r.get('entanglement_score'),
    'execution_counts': json.dumps(r.get('execution', {}).get('counts')) if r.get('execution') else None
} for r in results])
df.to_csv(csv_filename, index=False)